# Стохастические методы
Имитация очереди с покупателями, где задается ее размер, интенсивность прихода и интенсивность обслуживания. Возвращается статистика по среднему времени пребывания в очереди, количеству людей там, вероятности ожидания, вероятности потери покупателя из-за переполнения очереди

In [210]:
from scipy.stats import expon
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [35]:
class Visitor(object):
    def __init__(self, arriving):
        self.arriving :int = arriving
        self.start_serving : int = 0
        self.end_serving : int = 0
        
    def __repr__(self):
        return f"\nVisitor {self.arriving, self.start_serving, self.end_serving}"

In [176]:
class Queue(object):
    def __init__(self, size, intensity_arriving, intensity_serving):
        self.size = size
        self.intensity_arriving = intensity_arriving
        self.intensity_serving = intensity_serving
        self._visitors : list = []
        self.served = 0
        self.lost = 0
        self.waited = 0
        self.total_time = 0
        self._number_of_visitors: list = [] #cкаолько людей в системе в каждый момент времени
        self.info_table = pd.DataFrame({"average time":[],"regection prob":[],
                                       "waiting prob ":[],"average visitors":[]})
        
    def __repr__(self):
        return f"Queue {self.size, self.intensity_arriving, self.intensity_serving, self.served,self.waited, self.lost, self.total_time} \
        \n{len(self._visitors), self._visitors}"
    
    def new_visitor(self, time):
        if len(self._visitors) >= self.size:
            self.lost += 1
        else:
            self._visitors.append(Visitor(time))
        if len(self._visitors) >= 2:
            self.waited += 1
#         тут я не учитываю, что одновременно может прийти новый посетитель и в эту же 
#         секунду закончить обслуживание тот, что перед ним, то есть пришедший не будет ждать, 
#         но я хз, как это проверить в этой реализации, да и событие маловероятно
        self.add_number_visitors()
    
    def serving(self, start_time, end_time):
        v = self._visitors[0]
        v.start_serving = start_time
        v.end_serving = end_time
        
    def delete_visitor(self):
        v = self._visitors[0]
        self.total_time += v.end_serving - v.arriving   
        #только когда посетитель ушел, мы добавляем его время ожидания к остальным
        #если это очень плохо, скажите, поправлю
        self.served += 1
        self._visitors.pop(0)
        self.add_number_visitors()
        
    def add_number_visitors(self):
        self._number_of_visitors.append(len(self._visitors))
        
    def return_info(self):
        if self.served == 0:
            saver = 1
        else:
            saver = 0 
            # чтобы избежать division by zero в вероятностях
        time_in_system = self.total_time / (self.served + saver)
        n_visitors = len(self._visitors)
        lost  = self.lost / (self.lost + self.served + saver + n_visitors)
        waited = self.waited / (self.lost + self.served + saver + n_visitors)
        aver_visitors = np.mean(self._number_of_visitors)
#         print(f"Average time in system {time_in_system}\nProbability of regection {lost}\nProbability of waiting {waited}\nVisitors in system {n_visitors}\nAverage number of visitors {aver_visitors}\n")
   
        cols = self.info_table.columns
        self.info_table = pd.concat([self.info_table, 
                                     pd.DataFrame({cols[0]:[time_in_system], cols[1]:[lost], 
                                    cols[2]: [waited], cols[3]:[aver_visitors]})],ignore_index=True)
        
    def return_table(self):
        return self.info_table
        
        

In [205]:
TIME  = 0
queue = Queue(5, 10, 5) #задаем размер очереди, интенсивность прихода, интенсивность обслуживания
queue.new_visitor(TIME)
start_service = TIME

arr_time = expon(scale=1/queue.intensity_arriving).rvs(size=1)[0]
serv_time = expon(scale=1/queue.intensity_serving).rvs(size=1)[0]

In [206]:
arr_time, serv_time

(0.1199056584930036, 0.38451265444812144)

In [207]:
iteration = 0 
n_iterations = 10
while iteration < n_iterations: 
    while (arr_time <= serv_time and iteration < n_iterations):
        TIME += arr_time
        iteration += 1
        queue.new_visitor(TIME)
        queue.return_info()
        print("NEW ", queue)
        serv_time -= arr_time
        arr_time = expon(scale=1/queue.intensity_arriving).rvs(size=1)[0]

    while (serv_time < arr_time and iteration < n_iterations):
        TIME += serv_time
        queue.serving(start_service, TIME)    
        print("SERVED ", queue)
        queue.delete_visitor()
        queue.return_info()

        iteration += 1
        arr_time -= serv_time
        serv_time = expon(scale=1/queue.intensity_serving).rvs(size=1)[0]

        if len(queue._visitors) == 0: #больше некого обрабатывать, просто ждем следующего прибытия
            TIME += arr_time
            arr_time = 0
        start_service = TIME

NEW  Queue (5, 10, 5, 0, 1, 0, 0)         
(2, [
Visitor (0, 0, 0), 
Visitor (0.1199056584930036, 0, 0)])
SERVED  Queue (5, 10, 5, 0, 1, 0, 0)         
(2, [
Visitor (0, 0, 0.38451265444812144), 
Visitor (0.1199056584930036, 0, 0)])
NEW  Queue (5, 10, 5, 1, 2, 0, 0.38451265444812144)         
(2, [
Visitor (0.1199056584930036, 0, 0), 
Visitor (0.41822631430421603, 0, 0)])
NEW  Queue (5, 10, 5, 1, 3, 0, 0.38451265444812144)         
(3, [
Visitor (0.1199056584930036, 0, 0), 
Visitor (0.41822631430421603, 0, 0), 
Visitor (0.44344818513604134, 0, 0)])
SERVED  Queue (5, 10, 5, 1, 3, 0, 0.38451265444812144)         
(3, [
Visitor (0.1199056584930036, 0.38451265444812144, 0.4633832649041928), 
Visitor (0.41822631430421603, 0, 0), 
Visitor (0.44344818513604134, 0, 0)])
NEW  Queue (5, 10, 5, 2, 4, 0, 0.7279902608593106)         
(3, [
Visitor (0.41822631430421603, 0, 0), 
Visitor (0.44344818513604134, 0, 0), 
Visitor (0.4979673484355476, 0, 0)])
SERVED  Queue (5, 10, 5, 2, 4, 0, 0.727990260859

In [208]:
queue.return_table()

,average time,regection prob,waiting prob,average visitors
0,0.000000,0.0,0.333333,1.500000
1,0.384513,0.0,0.500000,1.333333
2,0.384513,0.0,0.666667,1.500000
3,0.384513,0.0,0.750000,1.800000
4,0.363995,0.0,0.750000,1.833333
5,0.363995,0.0,0.800000,2.000000
6,0.341309,0.0,0.800000,2.000000
7,0.341309,0.0,0.833333,2.111111
8,0.341309,0.0,0.857143,2.300000
9,0.341309,0.0,0.875000,2.545455


In [209]:
queue._number_of_visitors #сколько чуваков в системе в каждый момент наблюдения

[1, 2, 1, 2, 3, 2, 3, 2, 3, 4, 5]